<h1><center>Суммаризация текстов с помощью алгоритма TextRank</center></h1>

На этом занятии мы познакомимся с задачей суммаризации текстов на естественном языке.

Задача суммаризации заключается в том, чтобы из исходного текста получить более короткий текст, который будет содержать в себе всю (или почти всю) информацию, которая была в исходном тексте.
То есть по тексту нужно получить его краткое содержание так, чтобы потерять как можно меньше информации.

Методы решения этой задачи обычно делят на две категории:
- Extractive Summarization $-$ алгоритмы, основанные на выделении наиболее информативных частей исходного текста (предложения, абзацы и т.д.) и составлении краткого содержания из них.
- Abstractive Summarization $-$ алгоритмы, генерирующие новый текст на основе исходного.

Сегодня мы познакомимся с методом *TextRank*, основанном на графах. Это популярный алгоритм Extractive Summarization.

In [6]:
!pip install -q nltk

In [1]:
from tqdm import tqdm_notebook as tqdm

In [5]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
import os
import numpy as np

In [7]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

In [14]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict

## Загрузка данных

Будем использовать данные из корпуса новостей CNN/DailyMail.

В рамках этого занятия будем использовать небольшую подвыборку из 300 текстов CNN.

Загрузим их из интернета и сохраним в папку.

In [9]:
DATA_DIR = './cnn_stories_short/'

In [ ]:
!wget https://www.dropbox.com/s/kofxrgod7kl720m/cnn_stories_short.zip
!mkdir cnn_data
!unzip cnn_stories_short.zip -d $DATA_DIR

In [11]:
!rm -r ./cnn_stories_short/__MACOSX

## Подготовка данных

Датасет состоит из исходных текстов и уже написанных кратких содержаниях к ним.
Готовые краткие содержания сохраним в переменную *summaries* и воспользуемся ими позднее для оценки качества получившегося алгоритма.

In [12]:
texts = []
summaries = []
for filename in os.listdir(DATA_DIR):
    with open(os.path.join(DATA_DIR,filename),'r') as input_file:
        all_texts = input_file.read().split('@highlight')
        texts.append(all_texts[0])
        summaries.append('. '.join(map(lambda x: x.strip(), all_texts[1:])))

#### Нам понадобятся:
* тексты, разбитые на предложения
* предложения, разбитые на токены
* тексты, разбитые предложения, которые разбиты на токены

In [15]:
sent_tokenized_texts = [sent_tokenize(text) for text in texts]
tokenized_sentences = [word_tokenize(sent) for text in texts for sent in sent_tokenize(text)]
tokenized_texts = [[word_tokenize(sent) for sent in text] for text in sent_tokenized_texts]

## Построение векторных представлений для текстов

Для алгоритма TextRank нам потребуется получить векторное представление для каждого предложения в тексте.

Будем использовать предобученные вектора Glove.

Загрузим модель:

In [16]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2023-12-20 13:12:30--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-12-20 13:12:30--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-12-20 13:12:30--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [17]:
word_embeddings = {}
with open('glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f.readlines():
        values = line.split()
        word = values[0]
        word_embeddings[word] = np.asarray(values[1:], dtype='float32')

Теперь нам нужно на основе векторов слов построить векторные представления для предложений.

Воспользуемся для этого классом TfidfEmbeddingVectorizer, который мы уже реализовывали на предыдущих занятиях.

Он вычисляет вектор для текста как взвешенное среднее векторов входящих в него слов с весами tf-idf.

In [18]:
class TfidfEmbeddingVectorizer:

    def __init__(self, embedding_model, dim=100):
        self.embedding_model = embedding_model
        self.word2weight = None
        self.dim = dim

    def fit(self, X):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        max_idf = np.max(tfidf.idf_)
        self.word2weight = defaultdict(lambda: max_idf, [(w,tfidf.idf_[i]) for w,i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([np.mean([self.embedding_model[w] * self.word2weight[w]
                                  for w in words if w in self.embedding_model] or [np.zeros(self.dim)], axis=0)
                         for words in X])

In [19]:
sentence_vectorizer = TfidfEmbeddingVectorizer(word_embeddings)
sentence_vectorizer = sentence_vectorizer.fit(tokenized_sentences)

## Построение графа

Для алгоритма *TextRank* нам понадобится построить из текста взвешенный граф.

Каждому предложению текста мы поставим в соответствие вершину графа.

Каждая пара вершин будет соединена ребром, а вес ребра будет равен расстоянию между векторными представлениями соответствующих предложений.

Для примера построим граф в виде матрицы расстояний для одного из текстов.
Выберем один текст и построим для него матрицу расстояний. В качестве метрики используем косинусное расстояние.

In [20]:
TEXT_NUM = 5

In [21]:
sentences = tokenized_texts[TEXT_NUM]

С помощью векторизатора получим вектора для всех предложений текста.

In [22]:
vectorized_sentences = sentence_vectorizer.transform(sentences)

Воспользуемся функцией cosine_similarity из scikit-learn для вычисления матрицы косинусных расстояний.

In [23]:
G = cosine_similarity(vectorized_sentences)

## Extractive Summarization $-$ TextRank

Теперь мы реализуем непосредственно метод суммаризации текстов. Он будет основан на алгоритме *PageRank*.

*PageRank* $-$ рекурсивный алгоритм, который оценивает важность каждой вершины в графе на основе её связей с другими вершинами. Изначально алгоритм использовался для оценки важности интернет-страниц для поисковых систем.

Адаптация этого алгоритма для суммаризации текстов называется *TextRank*.

Алгоритм последовательно проходит по всем вершинам в графе и по нижеприведенной формуле пересчитывает значения PageRank для каждой из них.

Это происходит до тех пор, пока процесс не стабилизируется, то есть значения *PageRank* для всех вершин не перестанут существенно меняться с каждой новой итерацией.

$$ G = (V,E) - граф $$
$$$$
$$ PageRank(w) = (1-d) +  d \sum_{u} \frac {PageRank(u)} {C(u)}$$

$$u\ -\ вершина\ графа,\ такая\ что\ (u,w) \in E$$
$$$$
$$d = 0,85\ -\ коэффициент\ затухания$$

In [24]:
import numpy as np
from scipy.sparse import csr_matrix

Напишем нашу собственную реализацию *PageRank*.

In [25]:
def page_rank(G, s = .85, maxerr = .0001):

    n = G.shape[0]
    A = csr_matrix(G,dtype=np.float)
    rsums = np.array(A.sum(1))[:,0]
    ri, ci = A.nonzero()
    A.data /= rsums[ri]

    sink = rsums==0
    ro, r = np.zeros(n), np.ones(n)
    while np.sum(np.abs(r-ro)) > maxerr:
        ro = r.copy()
        for i in range(0,n):
            Ii = np.array(A[:,i].todense())[:,0]
            # account for sink states
            Si = sink / float(n)
            # account for teleportation to state i
            Ti = np.ones(n) / float(n)
            r[i] = ro.dot( Ii*s + Si*s + Ti*(1-s) )
    return r/float(sum(r))

In [26]:
scores = page_rank(G)

<ipython-input-25-098055473a38>:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  A = csr_matrix(G,dtype=np.float)


In [27]:
scores

array([0.070099  , 0.061521  , 0.06522922, 0.0674415 , 0.0690819 ,
       0.06947275, 0.05693864, 0.06838653, 0.06645888, 0.06845123,
       0.06885072, 0.0682811 , 0.06640581, 0.06792763, 0.06545407])

Можно также использовать готовую реализацию алгоритма PageRank из библиотеки для работы с графами NetworkX.
Сравним нашу реализацию с реализацией NetworkX.

In [28]:
!pip install -q networkx

In [31]:
import networkx as nx

nx_graph = nx.from_numpy_array(G)
nx_scores = nx.pagerank(nx_graph)

In [ ]:
sentence_num = 7
print("Our implementation: {0}\nNetworkX implementation: {1}".format(scores[sentence_num],nx_scores[sentence_num]))

Our implementation: 0.043935970863796754
NetworkX implementation: 0.04393597086379673


Чтобы получить краткое содержание нужного нам текста, нам теперь достаточно отсортировать все входящие в него предложения по TextRank и выбрать те, у которых он максимальный.

In [32]:
ranked_sentences = sorted(((scores[i],s,i) for i,s in enumerate(sentences)), reverse=True)

Выведем 5 предложений с наибольшим TextRank.

In [33]:
SUMMARY_LEN  = 5

for i in range(SUMMARY_LEN):
    print(' '.join(ranked_sentences[i][1]))

( CNN ) -- Five days of gunbattles between the Indian army and separatist militants in Indian-administered Kashmir have left at least 25 dead -- eight Indian army troopers , including one officer , and 17 militants , the Indian military said Tuesday .
However , various NGOs and rights groups put the number of dead at twice the official count .
Kashmir has been in the throes of a violent separatist campaign for nearly two decades during which authorities say 43,000 people have been killed .
This month 's encounter between the Indian Army and the militants is the second longest in Kashmir this year .
The spokesman denied media reports that helicopter gunships and heavy weapons had been used by the army during these operations against the militants .


Теперь объединим все в одну функцию summarize, которая будет получать на вход текст, разбитый по предложениям и выдавать 5 предложений с наибольшим *TextRank*.

In [36]:
def summarize(sentences,summary_len=5):
    vectorized_sentences = sentence_vectorizer.transform(sentences)
    G = cosine_similarity(vectorized_sentences)
    nx_graph = nx.from_numpy_array(G)
    nx_scores = nx.pagerank(nx_graph)
    ranked_sentences = sorted(((nx_scores[i],s,i) for i,s in enumerate(sentences)), reverse=True)
    summary = []
    for i in range(SUMMARY_LEN):
        summary.append(' '.join(ranked_sentences[i][1]))
    return summary

In [37]:
summarize(tokenized_texts[5])

['( CNN ) -- Five days of gunbattles between the Indian army and separatist militants in Indian-administered Kashmir have left at least 25 dead -- eight Indian army troopers , including one officer , and 17 militants , the Indian military said Tuesday .',
 'However , various NGOs and rights groups put the number of dead at twice the official count .',
 'Kashmir has been in the throes of a violent separatist campaign for nearly two decades during which authorities say 43,000 people have been killed .',
 "This month 's encounter between the Indian Army and the militants is the second longest in Kashmir this year .",
 'The spokesman denied media reports that helicopter gunships and heavy weapons had been used by the army during these operations against the militants .']

Теперь мы можем получить краткие содержания для каждого из текстов в нашей выборке.

In [38]:
system_summaries = [summarize(text) for text in tqdm(tokenized_texts)]

<ipython-input-38-8f2452768a61>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  system_summaries = [summarize(text) for text in tqdm(tokenized_texts)]


  0%|          | 0/300 [00:00<?, ?it/s]

In [39]:
test_sentence_num = 5
extracted_sentences = [sent_tokenized_texts[test_sentence_num][i] for score,sentence,i in ranked_sentences][:5]

In [46]:
print(*sent_tokenized_texts[5], sep='\n')

(CNN) -- Five days of gunbattles between the Indian army and separatist militants in Indian-administered Kashmir have left at least 25 dead -- eight Indian army troopers, including one officer, and 17 militants, the Indian military said Tuesday.
An Indian army soldier lays a wreath during the funeral of a slain soldier, northeast of Srinagar on Tuesday.
Defense Minister A.K.
Antony, meeting with India's military chiefs in Delhi, reviewed the situation in the Himalayan region and told the Army to deal with the situation in the Himalayan region with "utmost firmness."
Kashmir has been in the throes of a violent separatist campaign for nearly two decades during which authorities say 43,000 people have been killed.
However, various NGOs and rights groups put the number of dead at twice the official count.
In Srinigar, Kashmir, Army spokesman Lt. Col. J.S.
Brar told CNN the Army was moving against the militants "based on sound intelligence inputs as well as human intelligence provided by ou

In [48]:
print(*extracted_sentences, sep='\n')

(CNN) -- Five days of gunbattles between the Indian army and separatist militants in Indian-administered Kashmir have left at least 25 dead -- eight Indian army troopers, including one officer, and 17 militants, the Indian military said Tuesday.
However, various NGOs and rights groups put the number of dead at twice the official count.
Kashmir has been in the throes of a violent separatist campaign for nearly two decades during which authorities say 43,000 people have been killed.
This month's encounter between the Indian Army and the militants is the second longest in Kashmir this year.
The spokesman denied media reports that helicopter gunships and heavy weapons had been used by the army during these operations against the militants.


## Метрика качества:

Существуют разные метрики оценки качества алгоритмов суммаризации. Подробнее с ними можно познакомитсья здесь.

Одна из самых популярных метрик в этой задаче $-$  $ROUGE_n$.

$Доля\ n$-$грамм\ из\ рефератов,\ вошедших\ в\ s:$
$$$$
$ ROUGE_n(S) = \frac{\sum_{r\in R} \sum_{w} [w \in s][w \in r]}{\sum_{r \in R} \sum_w [w \in r]}$
$$$$
$ Доля\ n-грамм\ самого\ близкого\ реферата,\ вошедших\ в\ s:$
$$$$
$ ROUGE_{n_{multi}}(S) = \frac{max_{r\in R} \sum_{w} [w \in s][w \in r]}{\sum_{r \in R} \sum_w [w \in r]}$

Посмотрим, с каким качеством работает наш алгоритм.
Воспользуемся готовой реализацией метрики из библиотеки pyrouge: инструкция по установке [отсюда](https://stackoverflow.com/questions/45894212/installing-pyrouge-gets-error-in-ubuntu)

In [64]:
#!git clone https://github.com/bheinzerling/pyrouge
!pip install -e pyrouge

Obtaining file:///content/pyrouge
  Preparing metadata (setup.py) ... done
  Running setup.py develop for pyrouge


In [65]:
!git clone https://github.com/andersjo/pyrouge.git rouge

Cloning into 'rouge'...
remote: Enumerating objects: 393, done.
remote: Total 393 (delta 0), reused 0 (delta 0), pack-reused 393
Receiving objects: 100% (393/393), 298.74 KiB | 1.40 MiB/s, done.
Resolving deltas: 100% (109/109), done.


In [83]:
!pwd

/content


In [84]:
!pyrouge_set_rouge_path /content/rouge/tools/ROUGE-1.5.5/

2023-12-20 13:46:27,114 [MainThread  ] [INFO ]  Set ROUGE home directory to /content/rouge/tools/ROUGE-1.5.5/.


In [ ]:
!apt-get install libxml-parser-perl

In [86]:
! rm rouge/tools/ROUGE-1.5.5/data/WordNet-2.0.exc.db

In [111]:
%%bash
DIR="/content/rouge/tools/ROUGE-1.5.5"
pyrouge_set_rouge_path $DIR
cd $DIR/data
mv WordNet-2.0.exc.db WordNet-2.0.exc.db.orig
perl WordNet-2.0-Exceptions/buildExeptionDB.pl ./WordNet-2.0-Exceptions ./smart_common_words.txt ./WordNet-2.0.exc.db
python -m pyrouge.test

/tmp/tmpzyguak44/config_test.xml data/config_test.xml


2023-12-20 13:57:20,483 [MainThread  ] [INFO ]  Set ROUGE home directory to /content/rouge/tools/ROUGE-1.5.5.
mv: cannot stat 'WordNet-2.0.exc.db': No such file or directory
2023-12-20 13:57:20,950 [MainThread  ] [INFO ]  Written ROUGE configuration to /tmp/tmp3pr2fekx/rouge_conf.xml
.2023-12-20 13:57:21,433 [MainThread  ] [INFO ]  Processing files in data/SL2003_models_plain_text.
2023-12-20 13:57:21,433 [MainThread  ] [INFO ]  Processing SL.P.10.R.A.SL062003-08.html.
2023-12-20 13:57:21,433 [MainThread  ] [INFO ]  Processing SL.P.10.R.A.SL062003-23.html.
2023-12-20 13:57:21,434 [MainThread  ] [INFO ]  Processing SL.P.10.R.A.SL062003-07.html.
2023-12-20 13:57:21,434 [MainThread  ] [INFO ]  Processing SL.P.10.R.A.SL062003-24.html.
2023-12-20 13:57:21,435 [MainThread  ] [INFO ]  Processing SL.P.10.R.A.SL062003-16.html.
2023-12-20 13:57:21,435 [MainThread  ] [INFO ]  Processing SL.P.10.R.A.SL062003-13.html.
2023-12-20 13:57:21,435 [MainThread  ] [INFO ]  Processing SL.P.10.R.A.SL062003-2

Создадим папки system_input_dir и model_input_dir для кратких содержаний, сгенерированных нашим алгоритмом и для правильных кратких содержаний соответственно.

In [49]:
!mkdir system_input_dir
!mkdir model_input_dir

Сохраним краткие содержания в соответствующие папки. В каждом файле будет один текст, в каждой строке $-$ одно предложение.

In [50]:
sent_tokenized_summaries = [sent_tokenize(s) for s in summaries]

In [51]:
for i, summary in enumerate(sent_tokenized_summaries):
    with open('./model_input_dir/summary_{0}.txt'.format(i), 'w') as f:
        f.writelines([sentence+'\n' for sentence in summary])

In [52]:
for i, summary in enumerate(system_summaries):
    with open('./system_input_dir/summary_{0}.txt'.format(i), 'w') as f:
        f.writelines([sentence+'\n' for sentence in summary])

Для использования библиотеки pyrouge данные нужно перевести в в специальный формат.
Создадим папки для новых данных (в формате, нужном для библиотеки) и переведем данные в этот формат.

In [53]:
!mkdir system_output_dir
!mkdir model_output_dir

In [ ]:
from pyrouge import Rouge155

Rouge155.convert_summaries_to_rouge_format('./system_input_dir', './system_output_dir')
Rouge155.convert_summaries_to_rouge_format('./model_input_dir', './model_output_dir')

In [89]:
Rouge155.write_config_static(
    './system_output_dir', 'summary_(\d+).txt',
    './model_output_dir', 'summary_#ID#.txt',
    './rouge_conf.config')

In [112]:
!pyrouge_evaluate_rouge_format_files -s system_output_dir -sfp 'summary_(\d+).txt' -m model_output_dir -mfp 'summary_#ID#.txt'

2023-12-20 13:57:56,218 [MainThread  ] [INFO ]  Written ROUGE configuration to /tmp/tmphvetxggt/rouge_conf.xml
2023-12-20 13:57:56,219 [MainThread  ] [INFO ]  Running ROUGE with command /content/rouge/tools/ROUGE-1.5.5/ROUGE-1.5.5.pl -e /content/rouge/tools/ROUGE-1.5.5/data -c 95 -2 -1 -U -r 1000 -n 4 -w 1.2 -a -m /tmp/tmphvetxggt/rouge_conf.xml
---------------------------------------------
1 ROUGE-1 Average_R: 0.52867 (95%-conf.int. 0.51132 - 0.54781)
1 ROUGE-1 Average_P: 0.15862 (95%-conf.int. 0.15151 - 0.16601)
1 ROUGE-1 Average_F: 0.24066 (95%-conf.int. 0.23102 - 0.25059)
---------------------------------------------
1 ROUGE-2 Average_R: 0.16573 (95%-conf.int. 0.14998 - 0.18229)
1 ROUGE-2 Average_P: 0.04937 (95%-conf.int. 0.04393 - 0.05534)
1 ROUGE-2 Average_F: 0.07490 (95%-conf.int. 0.06694 - 0.08321)
---------------------------------------------
1 ROUGE-3 Average_R: 0.07832 (95%-conf.int. 0.06697 - 0.08941)
1 ROUGE-3 Average_P: 0.02358 (95%-conf.int. 0.01943 - 0.02801)
1 ROUGE-3 

## Абстрактивная суммаризация с помощью BART

BART - это энкодер-декодер модель, обученная для решения задачи суммаризации текста. Мы можем воспользоваться ее реализацией из библиотеки Transformers:

In [113]:
!pip install -q transformers datasets evaluate rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [114]:
from transformers import pipeline
import evaluate

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Мы не будем тюнить модель, только посмотрим на инференс и вычисление метрики ROUGE:

In [118]:
sample_text = ' '.join(sent_tokenized_texts[5])
sample_text

'(CNN) -- Five days of gunbattles between the Indian army and separatist militants in Indian-administered Kashmir have left at least 25 dead -- eight Indian army troopers, including one officer, and 17 militants, the Indian military said Tuesday. An Indian army soldier lays a wreath during the funeral of a slain soldier, northeast of Srinagar on Tuesday. Defense Minister A.K. Antony, meeting with India\'s military chiefs in Delhi, reviewed the situation in the Himalayan region and told the Army to deal with the situation in the Himalayan region with "utmost firmness." Kashmir has been in the throes of a violent separatist campaign for nearly two decades during which authorities say 43,000 people have been killed. However, various NGOs and rights groups put the number of dead at twice the official count. In Srinigar, Kashmir, Army spokesman Lt. Col. J.S. Brar told CNN the Army was moving against the militants "based on sound intelligence inputs as well as human intelligence provided by 

Результат суммаризации из модели:

In [124]:
summary = (summarizer(sample_text, max_length=130, min_length=30, do_sample=False))

summary[0]['summary_text']

"Eight Indian army troopers, including one officer, and 17 militants killed, Indian military says. Kashmir has been in throes of separatist campaign for nearly two decades. India's Defense Minister A.K. Antony meets with India's military chiefs in Delhi."

Суммаризированный текст из датасета:

In [121]:
summaries[5]

'NEW: 25 dead in Kashmir gunfights between Indian troops, separatist militants. Incident comes after India accuses Pakistani troops of firing on Indian troops. Kashmir has been in throes of separatist campaign for 20 years'

In [127]:
rouge = evaluate.load("rouge")

In [134]:
#help(evaluate.load("rouge"))

In [132]:
rouge.compute(
    predictions=[summary[0]['summary_text']],
    references=[summaries[5]]
    )

{'rouge1': 0.37333333333333335,
 'rouge2': 0.21917808219178084,
 'rougeL': 0.32,
 'rougeLsum': 0.32}